## Step 2 - Climate App

In [1]:
# Now that you have completed your initial analysis, design a Flask API based on the queries that you have just developed.

In [2]:
# •••••••••••••••••••••••••••••••••••••••••••• Dependencies & Flask Setup ••••••••••••••••••••••••••••••••••••••••••••••••••• #

import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

app = Flask(__name__)

In [3]:
# Use FLASK to create your routes, home page, and available routes
@app.route("/")
def home():
    """Greetings. Here is a list of available API routes"""
    return(
        f"Available Routes: <br/>"
        f"/api/v1.0/precipitation <br/>"
        f"/api/v1.0/stations <br/>"
        f"/api/v1.0/tobs <br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    )

### Precipitation Route

In [4]:
#   * Convert the query results to a Dictionary using `date` as the key and `prcp` as the value.
#   * Return the JSON representation of your dictionary.
@app.route("/api/v1.0/precipitation")
def precipitation():
    """Date & Temp Query for 2017"""
    precip = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.date >= '2017-01-01').all()
    preciplist = []
    for precip in precip:
        precipdict = {}
        precipdict["Date"] = Measurement.date
        precipdict["TOBS"] = Measurement.tobs
        precip.append(precipdict)
    return jsonify(precip)

### Station Route

In [5]:
#   * Return a JSON list of stations from the dataset.
@app.route("/api/v1.0/stations")
def stations():
    """Returns a JSON list of station from the dataset"""
    
    stashres = session.query(Station.station).all()
    allstash = list(np.ravel(station_results))
    
    return jsonify(allstash)

### Tobs Route

In [6]:
#   * query for the dates and temperature observations from a year from the last data point.
#   * Return a JSON list of Temperature Observations (tobs) for the previous year.
@app.route("/api/v1.0/tobs")
def tobs():
    """Returns a JSON list of temperature observations (TOBS) from 2017"""

    tobsres = session.query(Measurement.tobs).filter(Measurement.date >= '2017-01-01').all()
    alltobs = list(np.ravel(tobsres))
    
    return jsonify(alltobs)

### Start/End Date Routes

In [7]:
#   * Return a JSON list of the min temp, the av temp, and the max temp for a given start or start-end range.
#   * When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.
#   * When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` for dates b/t start/end date inclusive.
@app.route("/api/v1.0/calc_temps/<start>")

def calc_temps(start='startdt'):
    """Calculations for all dates greater >= start date, using only start date"""
    
    startdt = datetime.strptime('2017-01-01', '%Y-%m-%d').date()
    startres = session.query(func.max(Measurement.tobs), \
                            func.min(Measurement.tobs),\
                            func.avg(Measurement.tobs)).\
                            filter(Measurement.date >= start_date) 
    
    sttobs = []
    for tobs in startres:
        tobs_dict = {}
        tobs_dict["TAVG"] = float(tobs[2])
        tobs_dict["TMAX"] = float(tobs[0])
        tobs_dict["TMIN"] = float(tobs[1])
        
        sttobs.append(tobs_dict)

    return jsonify(sttobs)

@app.route("/api/v1.0/calc_temps/<start>/<end>")

def calc_temps_2(start='start_date', end='end_date'):  
    """Calculations for values between start and end dates"""
    
    startdt = datetime.strptime('2017-01-01', '%Y-%m-%d').date()
    enddt = datetime.strptime('2017-12-31', '%Y-%m-%d').date()

    stendres=session.query(func.max(Measurement.tobs).label("max_tobs"), \
                      func.min(Measurement.tobs).label("min_tobs"),\
                      func.avg(Measurement.tobs).label("avg_tobs")).\
                      filter(Measurement.date.between(start_date , end_date))   

    stendtobs = []
    for tobs in stendres:
        tobs_dict = {}
        tobs_dict["TAVG"] = float(tobs[2])
        tobs_dict["TMAX"] = float(tobs[0])
        tobs_dict["TMIN"] = float(tobs[1])

        stendtobs.append(tobs_dict)
    
    return jsonify(stendtobs)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\janac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
